In [2]:
import pandas as pd

class Wahlkreis:
    def __init__(self, id, name, num_votes, results):
        self.id = id
        self.name = name
        self.num_votes = num_votes
        self.results = results
        
    def print_results(self):
        print(f"Wahlkreis: {self.name}, Stimmen gesamt: {self.num_votes}")
        print("Ergebnisse:")
        for partei, percentage in self.results['relative'].items():
            print(f"  {partei}: {percentage:.2%}")

def lese_wahlkreise_aus_csv(datei_pfad):
    """
    Liest Wahlkreisdaten aus einer CSV-Datei und erstellt Wahlkreis-Objekte.
    
    :param datei_pfad: Pfad zur CSV-Datei
    :return: Liste von Wahlkreis-Objekten
    """
    # CSV-Datei einlesen
    dataframe = pd.read_csv(datei_pfad)
    
    # Parteien aus den Spalten extrahieren (alle ab der 4. Spalte)
    parteien = dataframe.columns[3:]
    
    # Liste der Wahlkreis-Objekte erstellen
    wahlkreise = []
    for _, row in dataframe.iterrows():
        nummer = int(row['id'])
        name = row['wahlkreisname']
        gesamt = int(row['gesamt'])
        
        ergebnisse = {'absolute': {}, 'relative': {}}
        for partei in parteien:
            if not pd.isna(row[partei]):
                ergebnisse['absolute'][partei] = int(row[partei])
                ergebnisse['relative'][partei] = row[partei]/gesamt
            else:
                ergebnisse['absolute'][partei] = 0
                ergebnisse['relative'][partei] = 0.0
            
        wahlkreise.append(Wahlkreis(nummer, name, gesamt, ergebnisse))
    
    return wahlkreise

def get_similarity_matrix(wahlkreise):
    matrix = {}
    
    for v in wahlkreise:
        matrix[v.id] = {}
        for w in wahlkreise:
            similarity = sum(min(v.results['relative'][p], w.results['relative'][p]) for p in v.results['relative'])
            matrix[v.id][w.id] = similarity
            
    return matrix

In [3]:
# Beispiel zur Nutzung
datei_pfad = 'landtagswahl_niedersachsen_2022.csv'  # Ersetzen mit dem tatsächlichen Pfad
wahlkreise = lese_wahlkreise_aus_csv(datei_pfad)

similarity_matrix = get_similarity_matrix(wahlkreise)

In [4]:
wahlkreis = 1
similarity_list = sorted([w.id for w in wahlkreise], key=lambda x: similarity_matrix[wahlkreis][x])

for w in similarity_list:
    print(w, wahlkreise[w-1].name)
    print(similarity_matrix[wahlkreis][w])

67 Cloppenburg
0.68047464554298
82 Papenburg
0.6806481923991557
68 Vechta
0.6974792524473584
81 Meppen
0.7159524550000703
66 Cloppenburg-Nord
0.7320452633543685
73 Bersenbrück
0.7327638327311433
80 Lingen
0.7418471396700826
87 Wittmund/Inseln
0.7573415080803866
84 Leer/Borkum
0.7600187864031623
11 Salzgitter
0.7639677396909864
54 Bremervörde
0.7641687833606009
57 Geestland
0.7672072491694873
5 Gifhorn-Nord/Wolfsburg
0.7685227521386481
10 Wolfenbüttel-Süd/Salzgitter
0.7699670210153694
12 Göttingen/Harz
0.7702569888107057
79 Grafschaft Bentheim
0.77204576951933
44 Bergen
0.7730135289491599
86 Aurich
0.7787000986239799
22 Alfeld
0.785193048540465
42 Walsrode
0.7854585499357324
13 Goslar
0.7855328615776364
8 Helmstedt
0.7855807788475078
38 Nienburg/Schaumburg
0.7862786914968822
83 Leer
0.78849277174884
41 Diepholz
0.7893218960666268
58 Cuxhaven
0.7905767678890766
85 Emden/Norden
0.7906609850665435
4 Peine
0.7908205165340536
35 Bad Pyrmont
0.7912564064211047
69 Wilhelmshaven
0.7928026814729

In [5]:
import json

# Dateiname der JSON-Datei
datei_pfad = "landtagswahl_niedersachsen_2021_wahlkreis_vergleich.json"

# Speichern des 2D-Dictionaries
with open(datei_pfad, "w", encoding="utf-8") as file:
    json.dump(similarity_matrix, file, indent=4, ensure_ascii=False)

print(f"Die Datei wurde als {datei_pfad} gespeichert.")

Die Datei wurde als landtagswahl_niedersachsen_2021_wahlkreis_vergleich.json gespeichert.
